# 重采样
在处理地球观测卫星数据的相关工作中，地理投影以及将数据重采样到这些不同投影是更复杂的主题之一。
这就是巴特斯比（Battersby）所著的《地图投影》一书中对地图投影的描述：
> 地图投影是将角度（球面/椭球面）坐标转换为平面坐标的过程。所有地图投影都会在最终的平面坐标中引入失真（例如面积、角度、距离的变形）。理解引入了哪些、在哪里以及多少失真，对于空间计算、空间模式的视觉解释以及任何地图的一般美学都是重要的考虑因素。
<img src="http://gistbok.ucgis.org/sites/default/files/figure2-projections.png" width="450px"/>

<sub>致谢：Battersby, S. (2017)。地图投影。地理信息科学与技术知识体系（2017年第二季度版），John P. Wilson（主编）。DOI：<a href="http://gistbok.ucgis.org/bok-topics/map-projections">10.22224/gistbok/2017.2.7</a></sub>
为了简化本教程中的投影和重采样过程，我们可以将重采样视为将数据从一种投影映射到另一种投影。投影描述了我们球形地球的一种平面版本，这种版本更容易描述。不同的人或可视化工具可能更倾向于使用某种投影来查看数据。在比较多个数据源的数据时，将它们全部转换为相同的投影可能会更加方便。让我们用真实数据来探讨这一点。

In [ ]:
%run ../init_notebook.py
from satpy import Scene
from glob import glob

filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.load(['C06'])

In [ ]:
scn['C06'].attrs['area']

我们 Satpy 数据的 `'area'` 属性是一个特殊的 `AreaDefinition`，它定义了我们的数据所覆盖的地理区域。在 `Projection` 下，我们看到一个用于定义地球平面表示的投影参数 Python 字典。我们的 ABI 数据使用 `'geos'` 或地球静止卫星视图坐标系，其中位置在 X 和 Y 轴上以米为单位进行测量。你可以在 PROJ 网站 [这里](https://proj.org/operations/projections/geos.html) 了解更多关于此的内容。根据我们如何更改这些参数，我们最终可能会得到如下所示的地球视图，或者完全不同的结果。
<img src="https://proj.org/_images/geos.png" width="300"/>

或者，我们可以读取完全不同的投影数据，例如[兰伯特等角圆锥投影](https://proj.org/operations/projections/lcc.html)投影，并查看如下图所示的视图。
<img src="https://proj.org/_images/lcc.png" width="300"/>

我们拥有的投影取决于卫星以及谁向我们提供了数据。我们希望在输出中使用的投影取决于我们的最终目标是什么。我们是否希望将我们的数据与其他卫星传感器的数据进行比较？我们是否希望以对感兴趣区域失真更小的投影方式查看我们的数据？
除了地球静止投影信息外，我们的 `AreaDefinition` 还指定了该投影空间中的确切位置：左下角（以投影米为单位）、右上角以及行和列像素的数量。
我们已经加载了 'C06'，让我们也加载 'C05' 通道，并通过比较它们来获得更好的投影和数据比较体验。

In [ ]:
scn.load(['C05'])
scn['C05'].attrs['area']

In [ ]:
scn['C06'].attrs['area']

注意两个区域定义之间的尺寸（行和列）差异，范围的细微差别，以及投影参数完全相同的情况。这是因为这两个通道使用相同的投影（坐标系统），但它们的像素**分辨率**不同。通道5的每个像素在静止卫星投影中代表1公里，而通道6的每个像素代表2公里。
由于数组形状的差异，尝试将这些与常规数组操作进行比较会很困难。相反，我们可以操作和重采样数据，以便后续进行更简单的计算。我们可以使用 Satpy 的 `Scene.resample` 方法，该方法提供了多种重采样算法。我们将从使用极其简单的 '原生' 重采样器开始。

In [ ]:
new_scn = scn.resample(resampler='native')
new_scn['C05'].shape == new_scn['C06'].shape

重采样方法为每个之前拥有的DataArray生成了一个新的`Scene`对象，但这些对象被重采样到相同的区域或地区。默认情况下，它使用输入数据中分辨率最高的`AreaDefinition`（`scn.max_area()`）。在这种情况下，这是来自`C05`的1km区域。现在如果我们查看`C06`的区域，可以看到它也是1km分辨率。

In [ ]:
new_scn['C06'].attrs['area']

我们使用的 `原生` 重采样器有两种可能的操作：
1. 如果将数据重新映射到更高分辨率，复制每个像素以使形状匹配。2. 如果将数据重新映射到较低的分辨率，请对像素进行平均/聚合以使形状一致。
既然我们的两个已加载通道具有相同的形状，就可以轻松地进行比较。让我们计算两个通道之间的差异并绘制出来。请注意，我们使用的是重新采样的 `new_scn`，**而不是** 原始的 `scn` 对象。

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

diff = new_scn['C06'] - new_scn['C05']
diff.plot.imshow(vmin=-20, vmax=20)

当需要将来自同一仪器但分辨率不同的波段进行组合时，`native` 重采样器可能会非常有用。然而，它的局限性在于只能处理相同投影下的数据，并且仅适用于可以轻松复制或聚合的数据（例如 1km → 2km，但不能处理 375m → 1km）。
要求：
- 保留 Markdown 格式
- 保留代码、URL 和 HTML 标签
- 使用标准技术术语
- 仅输出翻译，不添加解释

要进行更复杂的重采样，我们需要依赖其他重采样算法，其中最简单的是用于最近邻重采样的 `nearest` 重采样器。让我们创建一个自定义的 AreaDefinition，以便使用我们自己的投影进行重采样。为此，我们将使用 Pyresample 的 `create_area_def` 工具函数。运行以下单元格（带有 `?` 的单元格）将打开一个新面板，显示 `create_area_def` 函数的文档。我们可以利用这些信息开始构建一个新的重采样区域。使用完毕后可以关闭该面板。

In [ ]:
from pyresample import create_area_def
create_area_def?

In [ ]:
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -95, 'lat_0': 25, 'lat_1': 35},
                          width=1000, height=1000,
                          area_extent=[-105, 20, -90, 40], units='degrees')
my_area

In [ ]:
new_scn = scn.resample(my_area)

从上面的区域定义打印输出可以看出，我们使用了[Lambert Conformal Conic](https://proj.org/operations/projections/lcc.html)投影创建了一个1000行乘以1000列的区域。"区域范围"告诉我们，在投影空间的米制坐标中，我们的左下角和右上角的位置分别在哪里。我们可以查看一些常见的区域属性，以获取关于我们所创建区域的更多信息，例如每个像素的分辨率。

In [ ]:
my_area.pixel_size_x, my_area.pixel_size_y

在覆盖该地理区域的1000x1000像素网格中，我们让每个像素在X轴方向代表1440米，在Y轴方向代表2157米。
让我们再次使用 Xarray 的绘图工具来看看这会是什么样子。我们将指定 `vmin` 和 `vmax` 的范围在 0% 到 100% 之间，作为色阶的良好初始范围，这符合反射率数据的传统范围。

In [ ]:
plt.figure()
new_scn['C05'].plot.imshow(vmin=0, vmax=100)

因此，仅用几行代码，我们就改变了数据的投影、分辨率和整体尺寸。更重要的是，最初分辨率不同的数据已被重新采样到相同的地理区域和分辨率，从而更便于处理。
## 动态区域
然而，我们之前创建的区域需要我们确切知道想要的地球区域。如果我们只知道部分信息，并希望用我们的数据来填补其余部分呢？`create_area_def` 函数也能处理这种情况，需要时会创建一个 `DynamicAreaDefinition`。让我们创建一个动态区域，其中我们已知投影方式以及每个像素的分辨率（5000米）。请注意每一步计算所需的时间与之前调用相比的变化。

In [ ]:
my_dynamic_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -95, 'lat_0': 25, 'lat_1': 35},
                                  resolution=5000)
my_dynamic_area

In [ ]:
dynamic_scn = scn.resample(my_dynamic_area)
dynamic_scn['C05'].shape

In [ ]:
plt.figure()
dynamic_scn['C05'].plot.imshow(vmin=0, vmax=100)

你应该注意到的是，对 `resample` 的调用比之前花费了更长时间。这是由于需要进行计算以确定结果区域必须有多大才能包含所有数据。
此外，绘图调用会花费更长时间，因为我们要求对更多数据进行重新采样并绘制更多数据。请记住，dask数组仅在需要时加载和计算数据；在此情况下，即区域计算和绘图时。
### 练习
**时间: 10 分钟**
使用以下单元格在不同投影上绘制ABI数据，调整投影参数，或使用动态区域对整个区域进行重采样。注意当投影参数和区域范围变化时输出的变化。在小尺度下，投影变化可能不易察觉。
一些投影建议：
1. `{'proj': 'merc', 'lon_0': -97.0}`2. `{'proj': 'lcc', 'lon_0': -95, 'lat_0': 45, 'lat_1': 55, 'lat_2': 65}`3. `{'proj': 'stere', 'lon_0': -105.0, 'lat_ts': 25}`
请注意不要创建过大的区域（过多像素），否则可能需要等待一段时间以完成处理。
如果时间允许的话，尝试将其他频道加载到 `scn` 对象中（我们目前有 C05 和 C06），调整区域的范围，尝试其他 matplotlib 功能，调整 vmin/vmax 参数，或研究其他投影方式。如需查看支持的投影方式及其选项的完整列表，请参阅 [PROJ 文档](https://proj.org/operations/projections/index.html)。

In [ ]:
custom_area = create_area_def('my_area', {'proj': 'merc', 'lon_0': -97.0},
                              width=1000, height=1000,
                              area_extent=[-115, 15, -90, 40], units='degrees')
custom_area

In [ ]:
custom_scn = scn.resample(custom_area)

In [ ]:
plt.figure()
custom_scn['C05'].plot.imshow(vmin=0, vmax=100)

## 极轨卫星数据带
到目前为止，我们一直在查看搭载于地球静止卫星上的ABI仪器的数据。这类数据通常位于固定网格或区域内，所有像素均匀分布。现在我们将查看极轨卫星的数据；NOAA-20（JPSS-01）卫星的VIIRS仪器。

In [ ]:
polar_scn = Scene(reader='viirs_sdr', filenames=glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*j01*.h5'))
polar_scn.available_dataset_names()

In [ ]:
polar_scn.load(['I04'])
polar_scn['I04']

与迄今为止我们所查看的ABI波段相比，上述`I04`波段的信息存在一些重要的差异。首先，VIIRS上的五个`I`，即影像分辨率波段，其空间分辨率为每个像素约375米。ABI仪器的最高分辨率为每个像素500米，且仅有一个通道（C02）达到该分辨率，其余通道的分辨率为1公里和2公里。VIIRS上的其他16个`M`，即中分辨率波段，分辨率为750米。这是大多数极轨卫星的一个主要优势：具有更高分辨率的更多波段。事实上，本教程中提供的VIIRS数据由于数据集体积庞大而受到限制。
另一个需要注意的关键点是，与极轨数据相关的 `'area'` 是一种称为 `SwathDefinition` 的新类型对象。`SwathDefinition` 是一个简单的容器，用于存储经度和纬度数组。当数据的像素间距是非均匀的，且无法用单个“网格”中的像素来描述时，就需要这种类型的地理定位定义。

In [ ]:
type(polar_scn['I04'].attrs['area'])

In [ ]:
lons, lats = polar_scn['I04'].attrs['area'].get_lonlats()
lons

让我们绘制这个 `I04` 波段，看看在不进行任何重采样的情况下它看起来是什么样子。

In [ ]:
plt.figure()
polar_scn['I04'].plot.imshow()

这张图像的边缘看起来有些奇怪。这种现象是由仪器的扫描模式引起的，被称为“蝴蝶结效应”，即连续扫描的像素在扫描带边缘重叠的现象。在VIIRS的情况下，这种现象也被称为“蝴蝶结删除”，因为重叠产生的重复像素实际上被从数据中移除。您还可能注意到图像被水平翻转（墨西哥西海岸位于图像右侧）；这是数据观测方式和绘图方式带来的副作用。
纠正这些类型效应的最常见方法是对数据进行重采样。让我们重新使用之前为ABI创建的`my_area`定义，但在这里用于VIIRS。

In [ ]:
my_area

In [ ]:
new_polar_scn = polar_scn.resample(my_area)

In [ ]:
plt.figure()
new_polar_scn['I04'].plot.imshow()

此图表展示了典型极轨卫星数据的一个缺点。尽管我们拥有更多更高分辨率的光谱波段，但它们很少频繁覆盖同一区域。我们一直在关注的德克萨斯州的火灾，其大部分活动和增长发生在大约4小时内。来自NOAA-20的VIIRS数据（轨道上两个VIIRS仪器之一）是该仪器唯一一次捕捉到火灾任何部分的过境数据。然而，ABI仪器却每分钟都会获取新的图像。
## 比较 ABI 和 VIIRS
既然我们已经知道可以获取该区域的ABI和VIIRS数据，让我们进行一些比较。首先，我们需要加载两种仪器相似的通道或波段。我们将查看来自ABI的C05（1.61µm）和C07（3.90µm）通道，以及来自VIIRS的I03（1.61µm）和I04（3.74µm）通道。C05和I03波段均为反射率波段（%），C07和I04波段均为亮温波段（K）。
我们还应该从相似的时间点加载数据。鉴于VIIRS数据的时间分辨率，我们必须选择与我们可用的VIIRS数据相同时间（~20:30Z）的ABI数据。我们不再使用之前使用的CONUS扇区数据，而是改用较小的中尺度扇区。
我们将重新创建所有用于提高清晰度的场景和区域。

In [ ]:
abi_scn = Scene(reader='abi_l1b', filenames=glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/OR_ABI-L1b-RadM1-M3C??_G16_s20181312030*.nc'))
abi_scn.load(['C05', 'C07'])

In [ ]:
viirs_scn = Scene(reader='viirs_sdr', filenames=glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*j01*t203*.h5'))
viirs_scn.load(['I03', 'I04'])

让我们比较每个频段的时间和大小：

In [ ]:
print("ABI: ", abi_scn['C05'].attrs['start_time'])
print(abi_scn['C05'].shape)
print(abi_scn['C07'].shape)
print("VIIRS: ", viirs_scn['I03'].attrs['start_time'])
print(viirs_scn['I03'].shape)
print(viirs_scn['I04'].shape)

我们可以看到，两个ABI波段的分辨率较低（像素更少），并且在VIIRS数据采集后约8秒观测到。由于ABI数据已经网格化到地理静止（`geos`）投影，让我们将VIIRS波段重采样到ABI的区域。我们将使用ABI `Scene` 的特殊 `max_area` 方法，获取已加载两个通道中分辨率最高的区域。我们还需要将ABI通道重采样到相同分辨率，并可以像之前一样使用简单的 `native` 重采样器。请注意，默认情况下，`resample` 方法会使用当前Scene的 `max_area()`。
我们还将使用 `dask.persist` 函数来计算到目前为止的延迟 Dask 操作，并将结果保留在内存中。这将避免将来重新计算这些操作，但会占用我们机器的一部分内存。通常不需要这样做，但由于我们可能需要多次显示这些数据，最好避免每次重新处理。我们将再次添加 Dask 的 `ProgressBar`，以了解每次计算所需的时间。

In [ ]:
import dask
from dask.diagnostics import ProgressBar

with ProgressBar():
    abi_max_scn = abi_scn.resample(resampler='native')
    abi_max_scn['C05'], abi_max_scn['C07'] = dask.persist(abi_max_scn['C05'], abi_max_scn['C07'])

    viirs_max_scn = viirs_scn.resample(abi_scn.max_area())
    viirs_max_scn['I03'], viirs_max_scn['I04'] = dask.persist(viirs_max_scn['I03'], viirs_max_scn['I04'])

print(abi_max_scn['C05'].shape)
print(viirs_max_scn['I03'].shape)

从打印的形状可以看出，我们的ABI和VIIRS数据具有相同的形状。现在让我们绘制每个场景（`C05` 和 `I03`）的1.61微米波段。

In [ ]:
plt.figure()
abi_max_scn['C05'].plot.imshow(vmin=0, vmax=100)

In [ ]:
plt.figure()
viirs_max_scn['I03'].plot.imshow(vmin=0, vmax=100)

我们现在已将来自两种不同仪器的数据置于同一网格区域上。这将简化我们希望进行的任何比较或未来工作。
### 练习:
**时间：10 分钟**
使用以下两个单元格，利用 xarray 的绘图工具绘制加载的 ABI 和 VIIRS 波段之间的差异（每个单元格一个差异图）。尝试使用 `vmin` 和 `vmax` 关键字参数，查看限制颜色条范围是否能揭示更多信息。请注意，Xarray 会根据数据尝试选择最佳颜色映射。您可能需要通过 `cmap='viridis'` 或 `cmap='RdBu_r'` 强制使用特定颜色映射，或尝试 [其他颜色映射](https://matplotlib.org/tutorials/colors/colormaps.html) 由 matplotlib 提供。
如果时间允许，尝试绘制两个波段的平均值 (均值)。

In [ ]:
# C05 - I03


In [ ]:
# C07 - I04


<details>
<summary>解决方案（禁止作弊）：C05 - I03</summary>
```
plt.figure()
diff = abi_max_scn['C05'] - viirs_max_scn['I03']
diff.plot.imshow(add_colorbar=True, vmin=-25, vmax=25, cmap='RdBu_r')
```

</details>




<details>
<summary>解决方案（禁止作弊）：C07 - I04</summary>
```
plt.figure()
diff = abi_max_scn['C07'] - viirs_max_scn['I04']
diff.plot.imshow(add_colorbar=True, cmap='viridis')
```

</details>



<details>
<summary>解决方案：(C07 + I04) / 2</summary>
```
plt.figure()
diff = (abi_max_scn['C07'] + viirs_max_scn['I04']) / 2.0
diff.plot.imshow(add_colorbar=True, cmap='viridis')
```

</details>

通过使用重采样，我们能够合并那些通常无法实现的数组。这为更多类型的分析以及可以开展的科学研究打开了大门。

## 裁剪与聚合
假设我们不想改变投影，而只是想“裁剪”数据的一部分或降低数据的分辨率。Satpy 提供了 `Scene.crop` 和 `Scene.aggregate` 方法来帮助我们实现这一点。
我们将从使用之前用过的相同 ABI Mesoscale Scene 开始。假设我们想要裁剪出一个特定区域，但只知道该区域的经度和纬度坐标。通过使用数据中的 `AreaDefinition` 信息，我们可以无需进行任何昂贵的重采样，直接裁剪出数据中的特定区域。

In [ ]:
abi_scn = Scene(reader='abi_l1b', filenames=glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/OR_ABI-L1b-RadM1-M3C??_G16_s20181312030*.nc'))
abi_scn.load(['C05'])

In [ ]:
crop_scn = abi_scn.crop(ll_bbox=[-102.0, 34.0, -100.0, 36.0])
crop_scn['C05'].shape

In [ ]:
plt.figure()
crop_scn['C05'].plot.imshow(cmap='viridis', vmin=0, vmax=100)

也许我们想看到整个图像，但不需要所有数据的最高分辨率。我们可以使用 `Scene.aggregate` 方法来减少整体数组的大小。在这里，我们可以说要对 `y` 和 `x` 维度中每个 8x8 像素取平均值（默认）。

In [ ]:
agg_scn = abi_scn.aggregate(y=10, x=10)
agg_scn['C05'].shape

In [ ]:
plt.figure()
agg_scn['C05'].plot.imshow(vmin=0, vmax=100)

聚合函数使用几种不同的函数来组合数据。假设我们不是使用 `mean`，而是想要 `max` 值。

In [ ]:
plt.figure()
agg_scn = abi_scn.aggregate(y=10, x=10, func='max')
agg_scn['C05'].plot.imshow(vmin=0, vmax=100)

我们现在已经完成了对`resample`（重采样）、`crop`（裁剪）和`aggregate`（聚合）方法的学习，这些方法用于对提供的数据进行处理和调整，使其达到我们希望分析的分辨率、尺寸和区域。

## 进一步研究
* [其他重采样算法][1]* [缓存重采样][2]* [存储区域以供重复使用][3] 
[1]: https://satpy.readthedocs.io/en/latest/api/satpy.html#module-satpy.resample[2]: https://satpy.readthedocs.io/en/latest/resample.html#caching-for-geostationary-data[3]: https://satpy.readthedocs.io/en/latest/resample.html#store-area-definitions